In [1]:
# %pip install transformers datasets evaluate emojis
# %pip install torch scikit-learn  # sagemaker studio LAB

In [2]:
# import os
import sys

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import pipeline
from transformers import Trainer, TrainingArguments
from transformers import logging
import evaluate

import datasets
from datasets import load_metric

import emojis

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, balanced_accuracy_score, classification_report

print('sys.executable:', sys.executable)
print('sys.version   :', sys.version.replace('\n', ''))
print('sys.path      :')
for x in sys.path:
    print('               ', x)

# os.environ['WANDB_DISABLED'] = 'true'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

if torch.cuda.is_available():
    print('cuda n:', torch.cuda.device_count())
    current = torch.cuda.current_device()
    print('current', current, torch.cuda.device(current))
else:
    print('CPU')


sys.executable: /home/studio-lab-user/.conda/envs/default/bin/python
sys.version   : 3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:58:50) [GCC 10.3.0]
sys.path      :
                /home/studio-lab-user/tamil/tamil_mix_sentiment_analysis
                /home/studio-lab-user/.conda/envs/default/lib/python39.zip
                /home/studio-lab-user/.conda/envs/default/lib/python3.9
                /home/studio-lab-user/.conda/envs/default/lib/python3.9/lib-dynload
                
                /home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages
                /home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/IPython/extensions
                /home/studio-lab-user/.ipython
cuda
cuda n: 1
current 0 <torch.cuda.device object at 0x7f8b78882d30>


In [3]:
print(torch.__version__)
print(transformers.__version__)

1.13.1+cu117
4.25.1


In [4]:
dataset = datasets.load_dataset('tamilmixsentiment')
type(dataset)
dataset

testset = dataset.pop('test')
testset

my_emojis = [list(emojis.get(x)) for x in dataset['train']['text']]
my_emojis = [y for x in my_emojis for y in x]
print(pd.Series(my_emojis).value_counts())

Found cached dataset tamilmixsentiment (/home/studio-lab-user/.cache/huggingface/datasets/tamilmixsentiment/default/0.0.0/887420eecaf868ac6c10990649e49d10467e4cd4dffb98a6f20e4fe7c58df390)


  0%|          | 0/3 [00:00<?, ?it/s]

🤣    108
🤔     98
🤩     91
🤘     70
🤗     41
🥰     39
🤦     16
🤙     15
🤟      9
🦁      5
🧐      5
🤞      5
🥳      4
🤓      4
🤖      4
🧡      4
🤪      4
🥁      4
🤫      3
🤭      3
🤒      3
🤢      3
🤯      3
🤨      3
🤝      3
🤕      3
🤐      3
🤑      2
🦸      2
🥂      2
🤚      2
🤮      2
🤜      2
🤳      1
🤷      1
🥺      1
🤥      1
🦆      1
🤬      1
🤛      1
🤠      1
🤤      1
🧨      1
🦂      1
🥵      1
🤧      1
dtype: int64


In [5]:
# make subset of the data
def red(ds):
    p = 0.1
    n = int(ds.num_rows * p)
    res = ds.shuffle(seed=2023).select(range(n))
    return res


dataset = datasets.DatasetDict({
    'train': red(dataset['train']),
    'validation': red(dataset['validation']),
})
dataset

print(dataset['train'].to_pandas()['label'].value_counts(normalize=True))
print(dataset['validation'].to_pandas()['label'].value_counts(normalize=True))

Loading cached shuffled indices for dataset at /home/studio-lab-user/.cache/huggingface/datasets/tamilmixsentiment/default/0.0.0/887420eecaf868ac6c10990649e49d10467e4cd4dffb98a6f20e4fe7c58df390/cache-e0dc15cb3461e70b.arrow
Loading cached shuffled indices for dataset at /home/studio-lab-user/.cache/huggingface/datasets/tamilmixsentiment/default/0.0.0/887420eecaf868ac6c10990649e49d10467e4cd4dffb98a6f20e4fe7c58df390/cache-ef0be8f2bf001be5.arrow


0    0.688438
1    0.125331
2    0.106796
3    0.044131
4    0.035305
Name: label, dtype: float64
0    0.674603
1    0.142857
2    0.095238
3    0.063492
4    0.023810
Name: label, dtype: float64


In [6]:
model_name = 'bert-base-multilingual-cased'


tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)
print(len(tokenizer))  # 119547
assert tokenizer.tokenize('🧡') == ['[UNK]']

tokenizer.add_tokens(list(set(my_emojis)))
print(len(tokenizer))  # 119593
assert tokenizer.tokenize('🧡') == ['🧡']


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

num_labels = len(set(dataset['train']['label']))
print('num_labels:', num_labels)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
)
model.num_labels
model.resize_token_embeddings(len(tokenizer))
print('model.device', model.device)
model.to(device)
print('model.device', model.device)

119547
119593
num_labels: 5


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

model.device cpu
model.device cuda:0


In [27]:
def tokenize_function(ds):
    res = tokenizer(
        ds['text'],
        padding=True,
        truncation=True,
        # max_length=512,
        # return_tensors='pt',
    )
    return res

# tokenized_dataset = dataset.map(tokenize_function, batched=True)
# tokenize_function(dataset['validation'])

tokenized_dataset_train = tokenize_function(dataset['train'])
tokenized_dataset_validation = tokenize_function(dataset['validation'])

print(type(tokenized_dataset_train))
datasets.Dataset(tokenized_dataset_train)

<class 'transformers.tokenization_utils_base.BatchEncoding'>


TypeError: Expected a pyarrow.Table or a datasets.table.Table object, but got {'input_ids': [[101, 139, 10240, 10147, 26319, 13277, 119, 119, 19387, 12955, 58864, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 21946, 10679, 10206, 12300, 25948, 13277, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 62825, 10369, 13375, 10113, 56510, 26536, 10147, 11084, 119, 119, 119, 119, 119, 62825, 10369, 25212, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 109499, 25388, 10129, 10911, 10330, 21602, 10483, 38560, 64785, 11008, 27689, 13859, 24

In [22]:
type(tokenized_dataset_train)
tokenized_dataset_train.keys()
print({k: v.device for k, v in tokenized_dataset_train.items()})
print(tokenized_dataset_train['input_ids'].device)

print({k: v.to(device).device for k, v in tokenized_dataset_train.items()})  # just temporary
print(tokenized_dataset_train['input_ids'].device)

tokenized_dataset_train_gpu = {k: v.to(device) for k, v in tokenized_dataset_train.items()}
print({k: v.device for k, v in tokenized_dataset_train_gpu.items()})

tokenized_dataset_validation_gpu = {k: v.to(device) for k, v in tokenized_dataset_validation.items()}
print({k: v.device for k, v in tokenized_dataset_train_gpu.items()})


{'input_ids': device(type='cpu'), 'token_type_ids': device(type='cpu'), 'attention_mask': device(type='cpu')}
cpu
{'input_ids': device(type='cuda', index=0), 'token_type_ids': device(type='cuda', index=0), 'attention_mask': device(type='cuda', index=0)}
cpu
{'input_ids': device(type='cuda', index=0), 'token_type_ids': device(type='cuda', index=0), 'attention_mask': device(type='cuda', index=0)}
{'input_ids': device(type='cuda', index=0), 'token_type_ids': device(type='cuda', index=0), 'attention_mask': device(type='cuda', index=0)}


In [21]:
tokenized_dataset_validation_gpu_ds = datasets.Dataset.from_dict(tokenized_dataset_validation_gpu)
print(tokenized_dataset_validation_gpu_ds['input_ids'])



tokenized_dataset_validation_gpu_ds = datasets.Dataset.from_dict(tokenized_dataset_validation_gpu)
tokenized_dataset_validation_gpu_ds.set_format('torch')

tokenized_dataset_validation_gpu_ds['input_ids']


[[101, 24453, 11008, 11531, 33458, 10478, 13013, 10853, 14946, 28335, 10483, 10321, 16724, 13599, 13723, 17463, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 36702, 10465, 30186, 12582, 10147, 34961, 28335, 61562, 20631, 21840, 13013, 31743, 10147, 19655, 47537, 18161, 15105, 18661, 35637, 10113, 119, 119, 119, 119, 119, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 29890, 10165, 10132, 177, 33729, 10703, 10478, 13013, 10706, 10465, 17776, 10113, 22038, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 34348, 51077, 108270, 10330, 20950, 13723, 10853, 53839, 16942, 10465, 119, 11023, 16441, 10170, 89957, 10170, 10354, 10911, 10330, 23447, 64785, 13196, 21907, 117, 82116, 53587, 69215, 21744, 21153, 20194, 66365, 31675, 18161, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 13497, 10362, 22987, 12871, 119, 119, 119, 119, 10923, 10321, 50135, 10138, 10189, 109

tensor([[  101, 24453, 11008,  ...,     0,     0,     0],
        [  101, 36702, 10465,  ...,     0,     0,     0],
        [  101, 29890, 10165,  ...,     0,     0,     0],
        ...,
        [  101, 12786, 49123,  ...,     0,     0,     0],
        [  101, 19561, 21570,  ...,     0,     0,     0],
        [  101, 30186, 21995,  ...,     0,     0,     0]])

In [9]:
metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    res = metric.compute(predictions=predictions, references=labels)
    print('EVALUANDO', flush=True)
    print(type(res), flush=True)
    print(res, flush=True)
    return res


training_args = TrainingArguments(
    output_dir='./results_bert',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    save_strategy='epoch',  # 'no'
    evaluation_strategy='steps',  # 'no'
    report_to='none'
)


my_weights = 1 / dataset['train'].to_pandas()['label'].value_counts(normalize=True)
my_weights = np.log(my_weights)
my_weights = my_weights.tolist()
my_weights


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get('labels')
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get('logits')
        # compute custom loss (suppose one has 3 labels with different weights)
        # loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
        loss_fct = torch.nn.CrossEntropyLoss(weight=torch.tensor(my_weights))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    
trainer = CustomTrainer(
    model=model,
    args=training_args,
    # train_dataset=tokenized_dataset['train'],
    # eval_dataset=tokenized_dataset['validation'],
    train_dataset=tokenized_dataset_train_gpu,
    eval_dataset=tokenized_dataset_validation_gpu,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [10]:
print('trainer.args.device:', trainer.args.device)

trainer.args.device: cuda:0


In [11]:
trainer.train()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3
  Number of trainable parameters = 177892613


KeyError: 2

In [ ]:
trainer.save_model('./results_bert_final_trained_2')

In [ ]:
# Evaluation
ts = testset.to_pandas()

pip = pipeline('sentiment-analysis', './results_bert_final_trained_2')

res = pip(ts['text'].tolist())
res
ts[['pred', 'prob']] = pd.DataFrame(res).values
ts
ts['pred'].value_counts()
ts['pred'].value_counts(normalize=True)

ts['pred_n'] = ts['pred'].str.replace('LABEL_', '').astype(int)

pd.crosstab(ts['label'], ts['pred'])
pd.crosstab(ts['label'], ts['pred_n'])

precision_recall_fscore_support(ts['label'], ts['pred_n'])
pd.DataFrame(precision_recall_fscore_support(ts['label'], ts['pred_n']))

precision_recall_fscore_support(ts['label'], ts['pred_n'], average='weighted')

print(classification_report(ts['label'], ts['pred_n']))